In [1]:
from __future__ import division

import pathlib
%matplotlib inline


# ignore warning
import warnings
warnings.filterwarnings('ignore')

import os, cv2
import tensorflow as tf
import numpy as np
from networks import build_aggasatt_joint

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# tf.config.experimental.set_visible_devices([], 'GPU')

In [3]:
vgg19_path = './Models/imagenet-vgg-verydeep-19.mat'
pretrain_model_path = './Models/srdplus-pretrained/'

# original = "./original"
# source = "./source/"
# target = "./target/"

original = "./test/"
source = "./source/"
target = "./target/"

In [4]:
with tf.variable_scope(tf.get_variable_scope()):
    input=tf.placeholder(tf.float32,shape=[None,None,None,3])
    shadow_free_image=build_aggasatt_joint(input,64,vgg19_path)

sess=tf.Session()
sess.run(tf.global_variables_initializer())
idtd_ckpt=tf.train.get_checkpoint_state(pretrain_model_path)
saver_restore=tf.train.Saver([var for var in tf.trainable_variables()])
print('loaded '+idtd_ckpt.model_checkpoint_path)
saver_restore.restore(sess,idtd_ckpt.model_checkpoint_path)




[i] Hypercolumn ON, building hypercolumn features ... 


Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Use keras.layers.AveragePooling2D instead.




loaded ./Models/srdplus-pretrained/model_best.ckpt
INFO:tensorflow:Restoring parameters from ./Models/srdplus-pretrained/model_best.ckpt


In [5]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (24,6)

imgs = [os.path.join(original,x) for x in os.listdir(original) if '.jpg' in x or '.png' in x]

print(imgs)

['./original\\1.jpg', './original\\10.jpg', './original\\100.jpg', './original\\101.jpg', './original\\102.jpg', './original\\103.jpg', './original\\104.jpg', './original\\105.jpg', './original\\106.jpg', './original\\107.jpg', './original\\108.jpg', './original\\109.jpg', './original\\11.jpg', './original\\110.jpg', './original\\111.jpg', './original\\112.jpg', './original\\113.jpg', './original\\114.jpg', './original\\115.jpg', './original\\116.jpg', './original\\117.jpg', './original\\118.jpg', './original\\119.jpg', './original\\12.jpg', './original\\120.jpg', './original\\121.jpg', './original\\122.jpg', './original\\123.jpg', './original\\124.jpg', './original\\125.jpg', './original\\126.jpg', './original\\127.jpg', './original\\128.jpg', './original\\129.jpg', './original\\13.jpg', './original\\130.jpg', './original\\131.jpg', './original\\132.jpg', './original\\133.jpg', './original\\134.jpg', './original\\135.jpg', './original\\136.jpg', './original\\137.jpg', './original\\138

In [ ]:
for img_path in imgs:

    # iminput = cv2.resize(cv2.imread(img_path, 1), (468, 351))
    iminput = cv2.resize(cv2.imread(img_path, 1), (1020, 1440))

    imoutput = sess.run(shadow_free_image,feed_dict={input:np.expand_dims(iminput/255.,axis=0)})
    imoutput = np.uint8(np.squeeze(np.minimum(np.maximum(imoutput[0],0.0),1.0))*255.0)

    imname = pathlib.Path(img_path).stem
    cv2.imwrite(os.path.join(source, f'{imname}.jpg'), iminput)
    cv2.imwrite(os.path.join(target, f'{imname}.jpg'), imoutput)
    cv2.waitKey(0)

In [ ]:
print("success")
